## LEKOUNDA NGOLO Mardochet Gedeon
### COOP MSC | DATA SCIENCE
### ASSIGNMENT 4 : DATA COLLECTION
#### 29-111-2025

In [1]:
url = 'https://ci.coinafrique.com/categorie/appartements?page=13'
# scrape data (number of rooms, number of bathroom, area, price, location, image_link) over 200 pages
# using selenium combined with BeautifulSoup
# Clean the data

In [2]:
#!pip install selenium

In [3]:
# import packages
from selenium import webdriver
from selenium.webdriver.common.by import By

In [4]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import re

In [5]:
# instantiate a Chrome options object
options = webdriver.ChromeOptions()
# set the options to use Chrome in headless mode (used for running the script in the background)
options.add_argument("--headless=new")
# initialize an instance of the Chrome driver (browser) in headless mode
driver = webdriver.Chrome(options=options)

In [6]:
base_url = "https://ci.coinafrique.com/categorie/appartements?page="
data = []

In [7]:
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

def scrape_page(page_number):
    url = f'https://ci.coinafrique.com/categorie/appartements?page={page_number}'
    driver.get(url)
    time.sleep(3)

    soup = bs(driver.page_source, "html.parser")

    # REAL CSS CLASS FOR EACH AD PREVIEW
    containers = soup.find_all("div", class_="card ad__card")

    data = []

    for container in containers:
        try:
            # Extract product link
            a_tag = container.find("a", href=True)
            if not a_tag:
                continue

            product_url = "https://ci.coinafrique.com" + a_tag["href"]

            # Open detailed page
            driver.get(product_url)
            time.sleep(2)
            ## Mr ABDUL W. time.sleep(2) ceci est extremement necessaire pour les requetes 
            ## en effet ici on donne un delais de 2s pour chaque requetes, cela permettra de 
            ## recuperer un maximum nombre de page ainsi evitera de surcharger le serveur

            soup_detail = bs(driver.page_source, "html.parser")

            # Title
            title_tag = soup_detail.find("h1")
            title = title_tag.get_text(strip=True) if title_tag else None
            # On revoie None comme valeur par defaut dans le cas ou on arrive pas a recuperer le titer
            # Price
            price_tag = soup_detail.find("p", class_="ad__price")
            if not price_tag:
                price_tag = soup_detail.find("span", class_="price")

            if price_tag:
                price_raw = price_tag.get_text(strip=True)
                price = price_raw.replace("CFA", "").replace(" ", "")
            else:
                price = None

            # full adress
            loc_tag = soup_detail.find("p", class_="ad__card-location")
            if loc_tag:
                location = loc_tag.get_text(strip=True)
            else:
                # backup
                loc2 = soup_detail.find("span", class_="location")
                location = loc2.get_text(strip=True) if loc2 else None

            # ==============================
            # Extract Rooms / Bathrooms
            # ==============================
            rooms = None
            bathrooms = None

            info_list = soup_detail.find_all("li", class_="center")

            for li in info_list:
                label = li.find_all("span")
                if len(label) >= 2:
                    name = label[0].get_text(strip=True).lower()
                    qt = label[1].get_text(strip=True)

                    if "pièces" in name:
                        rooms = qt
                    elif "bain" in name:
                        bathrooms = qt
            
            # ==============================
            # Extract first main image
            # ==============================
            img_tag = soup_detail.find("img")
            image_link = img_tag["src"] if img_tag else None

            data.append({
                "title": title,
                "price": price,
                "location": location,
                "rooms": rooms,
                "bathrooms": bathrooms,
                "image": image_link,
                "url": product_url
            })

        except Exception as e:
            print("Error on item:", e)
            continue
    print(len(containers))
    return data


In [8]:
datas = [] # liste vide 

for page in range(1, 3):
    print(f"Scraping page {page}")
    page_data = scrape_page(page)
    datas.extend(page_data)

driver.quit()

df = pd.DataFrame(datas)
print(df.head())
print("Nombre total d'entrées:", len(df))

#df.to_csv("coinafrique_apartments_raw.csv", index=False)

Scraping page 1
0
Scraping page 2
0
Empty DataFrame
Columns: []
Index: []
Nombre total d'entrées: 0
